In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from encode_data import *
from midi_data import *

In [3]:
from tqdm import tqdm

## Encode music21 stream to text representation and back

### Load midi data

In [4]:
path = Path('data/midi')
csv_path = path/'metadata'

In [5]:
import pandas as pd

In [40]:
df = pd.read_csv(path/'metadata/combined.csv', index_col=0); df.head()

,artist,bpm,genres,ht_bpm,ht_key,ht_mode,ht_time_signature,inferred_key,inferred_keyc,instruments,...,midi_keyc,midi_title,original_path,parts,seconds,section,song_url,source,time_signature,title
0,wayne-sharpe,128.0,NaN,128.0,C,1.0,4.0,C major,C major,"Piano,Piano",...,data/midi/transposed/hooktheory/pianoroll/w/wa...,yu-gi-oh3,data/midi/midi_sources/hooktheory/pianoroll/w/...,"intro,chorus",15.000000,chorus,https://www.hooktheory.com/theorytab/view/wayn...,hooktheory,4/4,yu-gi-oh-theme-song
1,weezer,NaN,NaN,128.0,C,NaN,4.0,NaN,NaN,NaN,...,NaN,My New Song,data/midi/midi_sources/hooktheory/pianoroll/w/...,intro-and-verse,NaN,intro-and-verse,https://www.hooktheory.com/theorytab/view/weez...,hooktheory,NaN,beverly-hills
2,wayne-sharpe,85.0,NaN,85.0,C,1.0,3.0,C major,C major,Piano,...,data/midi/transposed/hooktheory/pianoroll/w/wa...,yu-gi-oh,data/midi/midi_sources/hooktheory/pianoroll/w/...,"intro,chorus",25.411765,intro,https://www.hooktheory.com/theorytab/view/wayn...,hooktheory,3/4,yu-gi-oh-theme-song
3,woodentoaster,197.0,"Alt-Country,Hip-Hop/Rap",197.0,D,6.0,4.0,D minor,D minor,"Piano,Piano",...,data/midi/transposed/hooktheory/pianoroll/w/wo...,Nightmare Night,data/midi/midi_sources/hooktheory/pianoroll/w/...,"chorus,instrumental",9.746193,instrumental,https://www.hooktheory.com/theorytab/view/wood...,hooktheory,4/4,nightmare-night
4,weezer,93.0,Rock,93.0,A,1.0,4.0,F# minor,F# minor,"Piano,Piano",...,data/midi/transposed/hooktheory/pianoroll/w/we...,Weezer - Fall Together,data/midi/midi_sources/hooktheory/pianoroll/w/...,chorus,10.322581,chorus,https://www.hooktheory.com/theorytab/view/weez...,hooktheory,4/4,fall-together-


In [41]:
enr = 'data/midi/transposed/freemidi/genre-dance-eletric/Enrique Iglesias - Experiencia Religiosa.mid'

In [42]:
df.loc[df.midi_keyc == enr]

,artist,bpm,genres,ht_bpm,ht_key,ht_mode,ht_time_signature,inferred_key,inferred_keyc,instruments,...,midi_keyc,midi_title,original_path,parts,seconds,section,song_url,source,time_signature,title
20,Enrique Iglesias,114.0,dance,NaN,NaN,NaN,NaN,F major,F major,"Flute,Piano,Electric Bass,Electric Organ,Voice",...,data/midi/transposed/freemidi/genre-dance-elet...,NaN,NaN,NaN,244.210526,NaN,NaN,freemidi,4/4,Experiencia Religiosa


In [7]:
df_filtered = df.loc[df['time_signature'] == '4/4'][18000:19000]; df_filtered.shape

(1000, 22)

In [8]:
from data_sources import process_parallel

In [9]:
def transcribe_file(idxrow):
    idx,row = idxrow
    midi_file = row.midi_keyc
    if not midi_file: return idx,None
    out_file = Path(midi_file.replace('/transposed/', '/transcribed_allcomp/')).with_suffix('.txt')
    out_file.parent.mkdir(parents=True, exist_ok=True)
    if out_file.exists(): return idx,str(out_file)
    seq = midi2seq(midi_file)
    string_repr = seq2str(seq)
    with open(out_file, 'w') as tf:
        tf.write(string_repr)
    return idx,str(out_file)

In [ ]:
# for r in df_filtered.iterrows():
#     transcribe_file(r)

In [ ]:
process_parallel(transcribe_file, df_filtered.iterrows(), total=df_filtered.shape[0])

In [9]:
# f1 = '../data/midi/transcribedc_v1/hooktheory/pianoroll/w/wayne-sharpe/yu-gi-oh-theme-song/chorus_key.txt'
# with open(f1, 'r') as f: 
#     print(f.read())

In [51]:
tdf = pd.DataFrame(data={'transcribed_keyc_v1': list(transcribed_files.values())}, index=transcribed_files.keys())

In [52]:
merged_df = df.join(tdf, how='outer')

In [53]:
tdf.shape, df.shape, merged_df.shape

((11531, 1), (12703, 18), (12703, 19))

In [56]:
merged_df.to_csv('hooktheory_metadata_transcribed.csv'); merged_df.head()

,parts,ht_mode,ht_key,original_path,genres,seconds,section,midi_keyc,time_signature,ht_time_signature,ht_bpm,instruments,bpm,midi_title,song_url,artist,inferred_key,inferred_keyc,transcribed_keyc_v1
0,"['intro', 'chorus']",1.0,C,../data/midi/hooktheory/pianoroll/w/wayne-shar...,[],15.000000,chorus,../data/midi/transposed/hooktheory/pianoroll/w...,4/4,4,128,"['Piano', 'Piano']",128.0,yu-gi-oh3,https://www.hooktheory.com/theorytab/view/wayn...,wayne-sharpe,C major,C major,../data/midi/transcribedc_v1/hooktheory/pianor...
1,"['intro', 'chorus']",1.0,C,../data/midi/hooktheory/pianoroll/w/wayne-shar...,[],25.411765,intro,../data/midi/transposed/hooktheory/pianoroll/w...,3/4,3,85,['Piano'],85.0,yu-gi-oh,https://www.hooktheory.com/theorytab/view/wayn...,wayne-sharpe,C major,C major,NaN
2,['chorus'],6.0,D,../data/midi/hooktheory/pianoroll/w/what-a-day...,['Jazz'],10.000000,chorus,../data/midi/transposed/hooktheory/pianoroll/w...,4/4,4,96,"['Piano', 'Piano']",96.0,kiefer,https://www.hooktheory.com/theorytab/view/what...,what-a-day,A minor,A minor,../data/midi/transcribedc_v1/hooktheory/pianor...
3,"['verse', 'pre-chorus', 'chorus']",6.0,D,../data/midi/hooktheory/pianoroll/w/whiteflame...,"['J-Pop', 'Pop']",25.263158,chorus,../data/midi/transposed/hooktheory/pianoroll/w...,4/4,4,152,"['Piano', 'Piano']",152.0,Senbonzakura,https://www.hooktheory.com/theorytab/view/whit...,whiteflame,D minor,D minor,../data/midi/transcribedc_v1/hooktheory/pianor...
4,"['verse', 'pre-chorus', 'chorus']",6.0,D,../data/midi/hooktheory/pianoroll/w/whiteflame...,"['J-Pop', 'Pop']",12.631579,verse,../data/midi/transposed/hooktheory/pianoroll/w...,4/4,4,152,"['Piano', 'Piano']",152.0,Senbonzakura,https://www.hooktheory.com/theorytab/view/whit...,whiteflame,D minor,D minor,../data/midi/transcribedc_v1/hooktheory/pianor...
